## Elliptic Curve Diffie-Hellman (ECDH) in Python From Scratch

### Reference

- [Key Exchange: From Carrying Handcuffed Briefcases To Modern Cryptosystems](https://sefiks.com/2016/04/11/key-exchange-from-carrying-handcuffed-briefcases-to-modern-cryptosystems/)
- [Elliptic Curve Diffie-Hellman (ECDH) Explained](https://www.youtube.com/watch?v=GOTIcxvYE94&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=10)
- [Elliptic Curve Diffie-Hellman (ECDH) in Python From Scratch](https://www.youtube.com/watch?v=445Opx6U3Co&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=10)
- [Elegant Signatures with Elliptic Curve Cryptography](https://sefiks.com/2018/02/16/elegant-signatures-with-elliptic-curve-cryptography/)


In [ ]:
# y^2 = x^3 + a* x + b

In [4]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
     
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
     
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
     
    is_on_curve((x3, y3), p)
         
    return x3, y3
 
def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
     
def apply_double_and_add_method(G, k, p):
    target_point = G
     
    k_binary = bin(k)[2:] #0b1111111001
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
         
        # doubling - always
        target_point = add_points(target_point, target_point, p)
         
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
     
    is_on_curve(target_point, p)
     
    return target_point

In [5]:
# from :https://sefiks.com/2018/02/16/elegant-signatures-with-elliptic-curve-cryptography/
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)


#finite field
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [6]:
is_on_curve(G, p)

In [7]:
point_2G = add_points(G,G,p)

In [8]:
point_2G

(89565891926547004231252920425935692360644145829622209833684329913297188986597,
 12158399299693830322967808612713398636155367887041628176798871954788371653930)

In [10]:
tmp_point = G
for i in range(0,21):
    tmp_point = add_points(tmp_point, G, p)
    print(f"{i}G = {tmp_point}")

0G = (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
1G = (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
2G = (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018)
3G = (21505829891763648114329055987619236494102133314575206970830385799158076338148, 98003708678762621233683240503080860129026887322874138805529884920309963580118)
4G = (115780575977492633039504758427830329241728645270042306223540962614150928364886, 78735063515800386211891312544505775871260717697865196436804966483607426560663)
5G = (41948375291644419605210209193538855353224492619856392092318293986323063962044, 48361766907851246668144012348516735800090617714386977531302791340517493990618)
6G = (2126205

In [11]:
apply_double_and_add_method(G= G, k = 20, p= p)

(34773495056115281091786765947597603724784643419904767525769502836017890139287,
 8470533044743364938367028725608288731153024648869546164814808839694950063162)

In [ ]:
# Q  = k x G

## Key Generation

In [12]:
import random

In [13]:
# alice private key
ka = random.getrandbits(256)
 
# bob private key
kb = random.getrandbits(256)
 
# alice public key
Qa = apply_double_and_add_method(G = G, k = ka, p = p)
 
# bob public key
Qb = apply_double_and_add_method(G = G, k = kb, p = p)


In [15]:
Qa

(24115737043057797690525500763743238651601163144851086444691141537953952573900,
 7143523938903372935854487295639979964351323490464980267806591588826134870117)

In [16]:
Qb

(93292925314962139635214476803215080857442183183622702067566729526800355489721,
 11948918701854876659258753309274792663210407891339277511232956058480846778289)

In [17]:
# alice performs this calculation
Sa = apply_double_and_add_method(G = Qb, k = ka, p = p)
 
# bob performs this calculation
Sb = apply_double_and_add_method(G = Qa, k = kb, p = p)
 
assert Sa == Sb

In [18]:
Sa

(79368113804262027932105949903922309031772698423225162833928164718003070672470,
 54761915782264346934845630678762116439937132011102669214102198872222165056361)

In [20]:
Sb

(79368113804262027932105949903922309031772698423225162833928164718003070672470,
 54761915782264346934845630678762116439937132011102669214102198872222165056361)